In [20]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import ElasticNet
from sklearn.datasets import make_regression
import statsmodels.formula.api as smf


In [42]:
df = pd.read_csv('data/train_data.csv', delimiter=',')

cols = df.columns[3:-1]
numFeatures = len(cols)

df = df.dropna(subset=cols) #drop na cells 
df = df[df.bedrooms <= 15]
df = df[df.sqft_lot <= 1250000]
df = df[df.sqft_above <= 7000]
df = df[df.sqft_basement <= 3500]
df = df[df.sqft_lot15 <= 500000]

scores = []
for i in df['yr_renovated']:
    if i != 0:
        scores.append((i - 2015) + 82)
    else:
        scores.append(0)
df['renovation_score'] = scores

landvalue = []
view = list(df['view'])
water = list(df['waterfront'])
living = list(df['sqft_living'])
lot = list(df['sqft_lot'])
grade = list(df['grade'])
condition = list(df['condition'])
for i in range(len(water)):
    value = (view[i] + 1) * living[i]
    if water[i] == 1:
        value  = value * 2
    value = value / lot[i]
    landvalue.append(value)
df['landvalue'] = landvalue

centerpoint= (47.628591, -122.289796)  #center point of city center
dist_df = df.loc[:,['Unique_idx','lat','long','price']]
dist_df['dist_latLon'] = np.sqrt(np.square(dist_df['lat']- centerpoint[0]) + np.square(dist_df['long']-centerpoint[1]))
df['distFromSeattle'] = dist_df['dist_latLon']


holder = df['price']
df = df.drop(['price'], axis=1)
df['price'] = holder
df 

,Unique_idx,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovation_score,landvalue,distFromSeattle,price
0,7291,4058200630,20141002T000000,3,1.75,2190,7021,1.0,0,2,...,0,98178,47.5033,-122.232,2180,7155,0,0.935764,0.137979,353000
1,14835,3613600150,20150105T000000,3,2.50,2370,6840,2.0,0,0,...,0,98119,47.6503,-122.366,1590,4400,0,0.346491,0.079236,300523
2,15880,7170200080,20140617T000000,2,1.00,1230,3800,1.0,0,0,...,0,98115,47.6797,-122.292,1610,3800,0,0.323684,0.051157,435000
3,8812,9542300530,20141124T000000,4,2.25,2510,9963,1.0,0,0,...,0,98005,47.5973,-122.177,3110,9963,0,0.251932,0.117056,800000
4,17220,3626039028,20140818T000000,3,1.00,1160,7491,1.0,0,0,...,0,98177,47.7024,-122.359,1800,2267,0,0.154852,0.101178,417500
5,18565,7657000540,20150304T000000,4,1.00,1220,7980,1.5,0,0,...,0,98178,47.4924,-122.237,1210,7920,0,0.152882,0.146066,260000
6,335,5009600070,20141007T000000,4,2.50,1960,5238,2.0,0,0,...,0,98038,47.3483,-122.052,1800,5894,0,0.374189,0.367573,260000
7,6273,3905050280,20140819T000000,3,2.50,2060,4812,2.0,0,0,...,0,98029,47.5793,-122.002,1930,5264,0,0.428096,0.291987,533000
8,10979,1925059200,20150407T000000,4,2.50,2700,17853,2.0,0,0,...,0,98004,47.6463,-122.219,3790,16672,0,0.151235,0.072977,1557600
9,13225,424069010,20140721T000000,4,2.25,2470,17008,2.0,0,0,...,0,98075,47.5924,-122.048,2470,31798,0,0.145226,0.244489,625000


In [43]:
# train_df = pd.read_csv('./data/train_data.csv')
# val_df = pd.read_csv('./data/val_data.csv')
# sample_df = pd.read_csv('./data/sample_submission.csv')

percent = 0.7  #choose percent to split 
n= df_train2.shape[0] #number of rows in data set before splitting
s= math.floor(percent * n)
y= df_train2.iloc[0:s,-1]
X = df_train2.iloc[0:s,:-1]

y_test= df_train2.iloc[s:n,-1]
X_test = df_train2.iloc[s:n,:-1]

In [44]:
olsmod = sm.OLS(y, X)
olsres = olsmod.fit()
print(olsres.summary())

                                 OLS Regression Results                                
Dep. Variable:                  price   R-squared (uncentered):                   0.920
Model:                            OLS   Adj. R-squared (uncentered):              0.920
Method:                 Least Squares   F-statistic:                              6939.
Date:                Wed, 12 Feb 2020   Prob (F-statistic):                        0.00
Time:                        15:55:06   Log-Likelihood:                     -1.6359e+05
No. Observations:               12098   AIC:                                  3.272e+05
Df Residuals:                   12078   BIC:                                  3.274e+05
Df Model:                          20                                                  
Covariance Type:            nonrobust                                                  
                       coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------

In [46]:
X['price']=y  #need to add price as last col for statsmodel

est03 = smf.ols('price ~ bedrooms + bathrooms + sqft_living + sqft_lot + floors + landvalue + waterfront + view + condition + grade + sqft_above + sqft_basement + yr_built + zipcode + yr_renovated +renovation_score +distFromSeattle + np.power(distFromSeattle, 2)', X).fit()
#cols2 = ['              bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors','landvalue','waterfront', 'view', 'condition', 'grade', 'sqft_above','sqft_basement', 'yr_built', 'distFromSeattle', 'zipcode', 'lat', 'long', 'yr_renovated', 'renovation_score', 'sqft_living15', 'sqft_lot15', 'price']

est03.summary().tables[0]

Dep. Variable:,price,R-squared:,0.744
Model:,OLS,Adj. R-squared:,0.743
Method:,Least Squares,F-statistic:,2060.
Date:,"Wed, 12 Feb 2020",Prob (F-statistic):,0.00
Time:,15:55:34,Log-Likelihood:,-1.6321e+05
No. Observations:,12098,AIC:,3.265e+05
Df Residuals:,12080,BIC:,3.266e+05
Df Model:,17,,
Covariance Type:,nonrobust,,


In [54]:
df = pd.read_csv('data/val_data.csv', delimiter=',')

cols = df.columns[3:-1]
numFeatures = len(cols)

df = df.dropna(subset=cols) #drop na cells 
df = df[df.bedrooms <= 15]
df = df[df.sqft_lot <= 1250000]
df = df[df.sqft_above <= 7000]
df = df[df.sqft_basement <= 3500]
df = df[df.sqft_lot15 <= 500000]

scores = []
for i in df['yr_renovated']:
    if i != 0:
        scores.append((i - 2015) + 82)
    else:
        scores.append(0)
df['renovation_score'] = scores

landvalue = []
view = list(df['view'])
water = list(df['waterfront'])
living = list(df['sqft_living'])
lot = list(df['sqft_lot'])
grade = list(df['grade'])
condition = list(df['condition'])
for i in range(len(water)):
    value = (view[i] + 1) * living[i]
    if water[i] == 1:
        value  = value * 2
    value = value / lot[i]
    landvalue.append(value)
df['landvalue'] = landvalue

centerpoint= (47.628591, -122.289796)  #center point of city center
dist_df = df.loc[:,['Unique_idx','lat','long','price']]
dist_df['dist_latLon'] = np.sqrt(np.square(dist_df['lat']- centerpoint[0]) + np.square(dist_df['long']-centerpoint[1]))
df['distFromSeattle'] = dist_df['dist_latLon']
df

,Unique_idx,id,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15,renovation_score,landvalue,distFromSeattle
0,15544,1310430130,20141009T000000,4,2.75,2790,6600,2.0,0,0,...,2000,0,98058,47.4362,-122.109,2900,6752,0,0.422727,0.264010
1,17454,2540830020,20150401T000000,3,2.25,1630,6449,1.0,0,0,...,1986,0,98011,47.7275,-122.232,1620,7429,0,0.252752,0.114557
2,21548,8835770330,20140819T000000,2,1.50,2370,184231,2.0,0,0,...,2005,0,98045,47.4543,-121.778,3860,151081,0,0.012864,0.540659
3,3427,7732400490,20141105T000000,4,2.50,2270,7665,2.0,0,0,...,1986,0,98052,47.6612,-122.148,2450,8706,0,0.296151,0.145497
4,8809,2800031,20150401T000000,3,1.00,1430,7599,1.5,0,0,...,1930,0,98168,47.4783,-122.265,1290,10320,0,0.188183,0.152323
5,3294,686450490,20140929T000000,3,2.00,2240,11250,1.0,0,0,...,1968,0,98008,47.6371,-122.119,2200,12500,0,0.199111,0.171008
6,275,4215100060,20150320T000000,3,2.50,2653,4510,2.0,0,0,...,2006,0,98031,47.4145,-122.166,2653,4927,0,0.588248,0.247306
7,8736,7853301570,20150430T000000,4,2.50,3550,10968,2.0,0,0,...,2006,0,98065,47.5431,-121.886,3550,8583,0,0.323669,0.412747
8,6161,7977201845,20140514T000000,3,1.75,1600,6120,1.5,0,0,...,1924,0,98115,47.6847,-122.291,1670,4590,0,0.261438,0.056122
9,19832,2970800105,20150313T000000,4,2.50,2420,5244,2.0,0,0,...,2007,0,98166,47.4729,-122.350,1400,5250,0,0.461480,0.166926


In [69]:
X = df
yhat = list((est03.predict(X)).astype(int))
df_final = pd.DataFrame(columns = ['Unique_idx', 'yhat']) 
df_final['Unique_idx'] = df['Unique_idx']
df_final['yhat'] = yhat
df_final

,Unique_idx,yhat
0,15544,630814
1,17454,509813
2,21548,727943
3,3427,690881
4,8809,220638
5,3294,675244
6,275,580351
7,8736,690937
8,6161,602085
9,19832,576750
